$$\textbf{Lab 4: Exercise 1}$$

$\textbf{Function under consideration: }\;\; f(\mathbf{x}) = 400
x_1^2 + 0.004x_2^4$

$\textbf{Gradient of the function:}$ \\
$\begin{aligned}\nabla f(\mathbf{x}) &= [\frac{\partial f(\mathbf{x})}{\partial x_1}, \frac{\partial f(\mathbf{x})}{\partial x_2}]^\top \\&= [800x_1, 0.016 x_2^3]^\top \end{aligned}$

$\textbf{Gradient of the function:}$ \\
$\begin{aligned}\nabla^2 f(\mathbf{x}) &= \begin{bmatrix} f_{x_1 x_1} & f_{x_1 x_2}\\ f_{x_2 x_1} & f_{x_2 x_2} \end{bmatrix}\\ &= \begin{bmatrix} 800 & 0\\ 0 & 0.048x_2^2 \end{bmatrix}\end{aligned}$ 

where $f_{x_1 x_1} = \frac{\partial^2f(\mathbf{x})}{\partial x_1^2}, f_{x_1 x_2} = \frac{\partial^2f(\mathbf{x})}{\partial x_1 \partial x_2},  f_{x_2 x_1} = \frac{\partial^2f(\mathbf{x})}{\partial x_2 \partial x_1}, f_{x_2 x_2} = \frac{\partial^2f(\mathbf{x})}{\partial x_2^2}$

$\text{For Newtown's method, the scaling matrix $\mathbf{D}^k$  is the inverse of the Hessian matrix $\nabla^2 f(\mathbf{x})$.}$
$\text{For the gradient descent algorithm (with scaling using a diagonal matrix) with backtracking line search, the scaling matrix $\mathbf{D}^k$ is the diagonal approximation of the inverse of $\nabla^2 f(\mathbf{x})$, given as:}$ $\begin{aligned}
\mathbf{D}^k = \begin{bmatrix} \frac{1}{f_{x_1 x_1}} & 0\\ 0 & \frac{1}{f_{x_2 x_2}} \end{bmatrix}
\end{aligned}
$, where $f_{x_1 x_1} = \frac{\partial^2f(\mathbf{x})}{\partial x_1^2}$ and  $f_{x_2 x_2} = \frac{\partial^2f(\mathbf{x})}{\partial x_2^2}$

In [1]:
#Importing the packages
import numpy as np 
from scipy.linalg import sqrtm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
import pandas as pd

In [2]:
# Method to compute function value at input x 
def evalf(x):   
    assert type(x) is np.ndarray and len(x) == 2
    return (400*(x[0]**2)) + (0.004*(x[1]**4))

In [3]:
# Method to compute gradient of the function at input x 
def evalg(x):  
    assert type(x) is np.ndarray and len(x) == 2     
    return np.array([800*x[0], 0.016*(x[1]**3)]) 

In [4]:
# Method to find Hessian matrix
def evalh(x):
    assert type(x) is np.ndarray 
    assert len(x) == 2
    return np.array([[800,0],[0,0.048*(x[1]**2)]])

In [5]:
# Method to construct D_k matrix used in GD with scaling with backtracking line search 
def compute_D_k(x):
    assert type(x) is np.ndarray
    assert len(x) == 2   
    elem1 = evalh(x)[0][0]
    elem2 = evalh(x)[1][1]
    return np.diag([1/elem1,1/elem2])

In [6]:
# Method to construct D_k matrix used in Newton's method
def compute_D_k_newton(x):
    assert type(x) is np.ndarray
    assert len(x) == 2  
    return np.linalg.inv(evalh(x))

In [7]:
# Computing steplength in GD using backtracking line search without scaling 

def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma): #add appropriate arguments to the function 
    assert type(x) is np.ndarray and len(gradf) == 2 
    assert type(gradf) is np.ndarray and len(gradf) == 2 
    assert type(alpha_start) is float and alpha_start>=0.
    assert type(rho) is float and rho>=0.
    assert type(gamma) is float and gamma>=0. 
    alpha = alpha_start
    p = -gradf
    while evalf(np.add(x, alpha*p)) > np.add(evalf(x), gamma*alpha*np.dot(gradf.T,p)):
        alpha = rho*alpha
    return alpha

In [8]:
# Computing steplength in GD using backtracking line search without scaling

def compute_steplength_backtracking_scaled_direction(x, gradf, D_k, alpha_start, rho, gamma): #add appropriate arguments to the function 
    assert type(x) is np.ndarray and len(gradf) == 2 
    assert type(gradf) is np.ndarray and len(gradf) == 2 
    assert type(alpha_start) is float and alpha_start>=0. 
    assert type(rho) is float and rho>=0.
    assert type(gamma) is float and gamma>=0. 
    assert type(D_k) is np.ndarray and len(D_k) == 2
    alpha = alpha_start
    p = -gradf
    while evalf(np.add(x, alpha*np.dot(D_k,p))) > np.subtract(evalf(x), gamma*alpha*np.dot(np.dot(D_k,gradf), gradf)): 
        alpha = rho*alpha   
    return alpha

In [9]:
#line search type 
CONSTANT_STEP_LENGTH = 1
BACKTRACKING_LINE_SEARCH = 2
# D_k choice type
newton = 3
diagonal_approx = 4

In [18]:
# Code for gradient descent with scaling to find the minimizer without scaling

def find_minimizer_gd(start_x, tol, line_search_type, *args):
    #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
    assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
    assert type(tol) is float and tol>=0 
    x = start_x
    g_x = evalg(x)
    #initialization for backtracking line search
    if(line_search_type == BACKTRACKING_LINE_SEARCH):
        alpha_start = args[0]
        rho = args[1]
        gamma = args[2]
    k = 0
    while (np.linalg.norm(g_x) > tol): 
        if line_search_type == BACKTRACKING_LINE_SEARCH:
            step_length = compute_steplength_backtracking(x,g_x, alpha_start,rho, gamma) 
        elif line_search_type == CONSTANT_STEP_LENGTH:
            step_length = 1.0
        else:  
            raise ValueError('Line search type unknown. Please check!')   
        # Gradient descent steps   
        x = np.subtract(x, np.multiply(step_length,g_x))
        k += 1 
        g_x = evalg(x) 
        #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
    return x, k, evalf(x) 
 

In [17]:
# Code for gradient descent with scaling to find the minimizer with scaling

def find_minimizer_gdscaling(start_x, tol, line_search_type, D_k_type, *args):
    #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
    assert type(start_x) is np.ndarray and len(start_x) == 2 
    assert type(tol) is float and tol>=0 
    x = start_x
    g_x = evalg(x)  
    #initialization for backtracking line search
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
    k = 0
    while (np.linalg.norm(g_x) > tol): 
        if D_k_type == newton:
            d_k = compute_D_k_newton(x)
        elif D_k_type == diagonal_approx:
            d_k = compute_D_k(x)
        else:
            raise ValueError("Invalid argument.")
        if line_search_type == BACKTRACKING_LINE_SEARCH:
            step_length = compute_steplength_backtracking_scaled_direction(x, g_x, d_k, alpha_start, rho, gamma) 
        elif line_search_type == CONSTANT_STEP_LENGTH: 
            step_length = 1.0
        else:  
            raise ValueError('Line search type unknown. Please check!')       
        # Gradient descent steps
        x = np.subtract(x, np.multiply(step_length,np.dot(d_k, g_x))) 
        k += 1 
        g_x = evalg(x) 
        #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
    return x, k, evalf(x)

In [12]:
my_start_x = np.array([2.0,2.0])
my_tol= 1e-9
alpha_start = 1.
rho = 0.5
gamma = 0.5

#Q-3

In [13]:
# Gradient descent using Newton's method with constant step length
x_opt_newton_const, iter_newton_const, f_opt_newton_const = find_minimizer_gdscaling(my_start_x, my_tol, CONSTANT_STEP_LENGTH, newton, alpha_start, rho, gamma)
print(f"\nNewton's method (with constant step length of 1) for starting point {my_start_x}:\nMinimizer: {x_opt_newton_const}\nFinal Objective function value: {f_opt_newton_const}\nIterations taken to terminate: {iter_newton_const}")


# Gradient descent using Newton's method with backtracking line search
x_opt_newton_bls, iter_newton_bls, f_opt_newton_bls = find_minimizer_gdscaling(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, newton, alpha_start, rho, gamma)
print(f"\nNewton's method (with Backtracking line search) for starting point {my_start_x}:\nMinimizer: {x_opt_newton_bls}\nFinal Objective function value: {f_opt_newton_bls}\nIterations taken to terminate: {iter_newton_bls}") 


Newton's method (with constant step length of 1) for starting point [2. 2.]:
Minimizer: [0.         0.00304488]
Final Objective function value: 3.4382653805813626e-13
Iterations taken to terminate: 16

Newton's method (with Backtracking line search) for starting point [2. 2.]:
Minimizer: [0.         0.00304488]
Final Objective function value: 3.4382653805813626e-13
Iterations taken to terminate: 16


# *Comment:*
$\text{For the Newton's method with both constant step length as well as with backtracking line search, the algorithm terminated in 16 iterations, } \\ \text{attaining the same minimizer and the same minimum objective function value.} \\
\text{The minimizer obtained in both the cases is $(0, 0.0.00304488)$} \\
\text{The minimum objective function value at the obtained minimizer in both the cases came out to be $3.4382653805813626e-13$} \\
\text{We may say that the choice of the step-length did not affect the rate of convergence of the algorithm,} \\ \text{Rather the consideration of the scaling algorithm (with the scaling matrix taken as ($\nabla^2 f(\mathbf{x}))^{-1}$ in Newton's method) was more crucial.}$


#Q-4

In [16]:
# Gradient descent with backtracking line search without scaling
#x_opt_bls, iter_bls, f_opt_bls = find_minimizer_gd(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, alpha_start, rho, gamma)
#print(f"\nGD (without scaling) with backtracking line search for starting point {my_start_x}:\nMinimizer: {x_opt_bls}\nFinal Objective function value: {f_opt_bls}\nIterations taken to terminate: {iter_bls}")

Streaming output truncated to the last 5000 lines.
iter: 32598  x: [2.38357772e-06 5.02036193e-01]  f(x): 0.000254099603267875  grad at x: [0.00190686 0.00202453]  gradient norm: 0.0027811618257381444
iter: 32599  x: [-1.34076246e-06  5.02032239e-01]  f(x): 0.0002540900444957751  grad at x: [-0.00107261  0.00202449]  gradient norm: 0.0022910775230506296
iter: 32600  x: [2.84912024e-06 5.02024331e-01]  f(x): 0.0002540765628731631  grad at x: [0.0022793  0.00202439]  gradient norm: 0.0030484992757265263
iter: 32601  x: [-1.60263013e-06  5.02020377e-01]  f(x): 0.00025406633913028164  grad at x: [-0.0012821   0.00202434]  gradient norm: 0.0023961957342857336
iter: 32602  x: [3.40558903e-06 5.02012469e-01]  f(x): 0.0002540539436856374  grad at x: [0.00272447 0.00202425]  gradient norm: 0.0033941595784664734
iter: 32603  x: [7.44972601e-07 5.02010493e-01]  f(x): 0.0002540455249494322  grad at x: [0.00059598 0.00202422]  gradient norm: 0.0021101347915540012
iter: 32604  x: [-8.56718491e-06  5

KeyboardInterrupt: ignored

In [19]:
# Gradient descent using backtracking line search with scaling using diagonal matrix
x_opt_scaled_bls, iter_scaled_bls, f_opt_scaled_bls = find_minimizer_gdscaling(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, diagonal_approx, alpha_start, rho, gamma)
print(f"\nGD (with scaling using diagonal matrix) with backtracking line search for starting point {my_start_x}:\nMinimizer: {x_opt_scaled_bls}\nFinal Objective function value: {f_opt_scaled_bls}\nIterations taken to terminate: {iter_scaled_bls}")


GD (with scaling using diagonal matrix) with backtracking line search for starting point [2. 2.]:
Minimizer: [0.         0.00304488]
Final Objective function value: 3.4382653805813626e-13
Iterations taken to terminate: 16


#*Comment:*
$\textbf{Regarding rate of termination:} 
\; \; \text{For the gradient descent algorithm (with scaling using a diagonal matrix) with backtracking line search, the algorithm terminated in 16 iterations,}\\ \text{ same as the number of iterations taken in both the variants of the Newton's method. }
\; \; \text{The gradient descent algorithm (without scaling) with backtracking line search terminated in 37597 iterations, tremendously higher than any of the other cases.} \\
\textbf{Regarding obtained minimizer:}
\; \; \text{The minimizer obtained in the gradient descent algorithm (with scaling using a diagonal matrix) with backtracking line search is (0, 0.0.00304488),} \\ \text{ same as that obtained in the 2 variant of the Newton's method.} 
\; \; \text{The minimizer obtained in the gradient descent algorithm (without scaling) with backtracking line search is (-7.43026176e-07  4.69406653e-01)} \\
\textbf{Regarding minimum objective function value at the obtained minimizer:}
\; \; \text{The function value at the obtained minimizer in the gradient descent algorithm (with scaling using a diagonal matrix) with backtracking line search comes out to be 3.4382653805813626e-13,} \\ \text{ same as that obtained in both the variants of the Newton's method.}
\; \; \text{The function value at the obtained minimizer in the gradient descent algorithm (without scaling) with backtracking line search comes out to be 0.00019420367607344131} \\ \text{ Thus, we may say that scaling plays a very important role in the termination rate of the algorithm, significantly reducing the required number of iterations. The minimizer and the minimum function value came out to be almost the same, but not exactly the same.}$